In [1]:
import pandas as pd

from os.path import join

In [2]:
DATA_PATH = "data_dir"

## Тестим nmd-escape из статьи Torene et al., 2024  
(https://doi.org/10.1016/j.ajhg.2023.11.007)

https://github.com/rebeccaito/nmd-escape/tree/main

In [2]:
from annotating_nmd import * 

In [3]:
cds_bed_df = pd.read_table('nmd-escape/tests/data/test_cds.bed', names=['chrom', 'start', 'end', 'cds_id', 'score', 'strand'])
cds_bed_df

,chrom,start,end,cds_id,score,strand
0,chr17,58677775,58678247,NM_003620.4_cds_0_0_chr17_58677776_f,0,+
1,chr17,58700881,58701110,NM_003620.4_cds_1_0_chr17_58700882_f,0,+
2,chr17,58711213,58711338,NM_003620.4_cds_2_0_chr17_58711214_f,0,+
3,chr17,58725252,58725443,NM_003620.4_cds_3_0_chr17_58725253_f,0,+
4,chr17,58733959,58734202,NM_003620.4_cds_4_0_chr17_58733960_f,0,+
5,chr17,58740355,58740913,NM_003620.4_cds_5_0_chr17_58740356_f,0,+
6,chr14,99640487,99642532,NM_138576.4_cds_0_0_chr14_99640488_r,0,-
7,chr14,99697681,99697894,NM_138576.4_cds_1_0_chr14_99697682_r,0,-
8,chr14,99723807,99724176,NM_138576.4_cds_2_0_chr14_99723808_r,0,-
9,chr14,99737497,99737555,NM_138576.4_cds_3_0_chr14_99737498_r,0,-


#### 1. Determine NMD regions from a 6-column bed file of coding sequence regions

Note: the cds_id column must follow the conventions for RefSeq transcript CDS downloaded from the UCSC Genome Browser: e.g., `NM_138576.4_cds_0_0_chr14_99640488_r` is a valid `cds_id`

In [4]:
nmd_bed_df = make_boundaries_df(cds_bed_df)
nmd_bed_df

index  chrom     start       end  \
transcript_name                                      
NM_003620.4     0     5  chr17  58740355  58740913   
                1     4  chr17  58734147  58734202   
NM_138576.4     0     6  chr14  99640487  99642532   
                1     7  chr14  99697681  99697736   

                                                 cds_id score strand  \
transcript_name                                                        
NM_003620.4     0  NM_003620.4_cds_5_0_chr17_58740356_f     0      +   
                1  NM_003620.4_cds_4_0_chr17_58733960_f     0      +   
NM_138576.4     0  NM_138576.4_cds_0_0_chr14_99640488_r     0      -   
                1  NM_138576.4_cds_1_0_chr14_99697682_r     0      -   

                  transcript_name cds_size  
transcript_name                             
NM_003620.4     0     NM_003620.4      558  
                1     NM_003620.4       55  
NM_138576.4     0     NM_138576.4     2045  
                1     NM_138576.4       55

#### 2. Determine NMD size per transcript

In [5]:
sizes_df = make_cds_size_df(cds_bed_df)
sizes_df

/home/bioinf/gnomad-variants/Ivanova/nmd-escape/annotating_nmd/__init__.py:183: FutureWarning: The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.
  cds_sizes = bed_df.groupby('transcript_name').cds_size.apply(sum).reset_index().rename(columns={0: 'cds_size'})


,transcript_name,cds_size,nmd_escape_size,total_pdot_length,nmd_pdot_start
0,NM_003620.4,1818,613,606,401.666667
1,NM_138576.4,2685,2100,895,195.000000


#### 3. Determine whether frameshifts would be NMD escaping

For frameshifts and stop gain mutations that occur within the NMD(-) region, there are several variant annotators that will note the variant as NMD escaping. There are situations, however, where a frameshift variant can occur upstream and introduce a stop codon within the NMD(-) region. Below is an example of how one could use HGVSp notation from a typical variant annotator, like VEP or Jannovar, parse where the stop codon is introduced, and determine whether NMD escape is occurring.

In [31]:
sizes_df = make_cds_size_df(cds_bed_df)
# variant annotations (from VEP for example)
data = [
    {
        'transcript_name': 'NM_003620.4',
        'HGVSp': 'NP_003611.1:p.Cys300LeufsTer129',
    }, {
        'transcript_name': 'NM_003620.4',
        'HGVSp':'NP_003611.1:p.Cys81LeufsTer129',
    }
]
variant_df = pd.DataFrame(data)
result = get_upstream_frameshift(variant_df, sizes_df)
result

/home/bioinf/gnomad-variants/Ivanova/nmd-escape/annotating_nmd/__init__.py:183: FutureWarning: The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.
  cds_sizes = bed_df.groupby('transcript_name').cds_size.apply(sum).reset_index().rename(columns={0: 'cds_size'})


,transcript_name,HGVSp,var_pdot,stop_pdot_shift,stop_pdot,nmd_pdot_start,is_nmd_frameshift
0,NM_003620.4,NP_003611.1:p.Cys300LeufsTer129,300,129,429,401.666667,True
1,NM_003620.4,NP_003611.1:p.Cys81LeufsTer129,81,129,210,401.666667,False


### Создадим bed-файл из нужной аннотации

! `zcat gencode.v45.annotation.gtf.gz | awk 'OFS="\t" {if ($3=="CDS") {print $1,$4-1,$5,$12,$6,$7}}' | tr -d '";' > gencode.v45.annotation.bed`

In [6]:
cds_bed = pd.read_table('data_dir/gencode.v45.annotation.bed', names=['chrom', 'start', 'end', 'cds_id', 'score', 'strand'])

In [9]:
cds_bed['cds_id'] = (cds_bed['cds_id'] + '_cds_0_0_' +
                              cds_bed['chrom'] + '_' +
                              (cds_bed['start'] + 1).astype(str) + '_' +
                              cds_bed['strand'].map({'+': 'f', '-': 'r'}))

In [10]:
cds_bed

,chrom,start,end,cds_id,score,strand
0,chr1,65564,65573,ENST00000641515.2_cds_0_0_chr1_65565_f,.,+
1,chr1,69036,70005,ENST00000641515.2_cds_0_0_chr1_69037_f,.,+
2,chr1,450742,451678,ENST00000426406.4_cds_0_0_chr1_450743_r,.,-
3,chr1,685718,686654,ENST00000332831.5_cds_0_0_chr1_685719_r,.,-
4,chr1,924431,924948,ENST00000616016.5_cds_0_0_chr1_924432_f,.,+
...,...,...,...,...,...,...
885744,chrM,10469,10763,ENST00000361335.1_cds_0_0_chrM_10470_f,.,+
885745,chrM,10759,12137,ENST00000361381.2_cds_0_0_chrM_10760_f,.,+
885746,chrM,12336,14145,ENST00000361567.2_cds_0_0_chrM_12337_f,.,+
885747,chrM,14148,14673,ENST00000361681.2_cds_0_0_chrM_14149_r,.,-


Determine NMD regions from a 6-column bed file of coding sequence regions:

In [11]:
nmd_bed = make_boundaries_df(cds_bed)

In [12]:
nmd_bed

index  chrom      start        end  \
transcript_name                                             
ENST00000000233.10 0  358020   chr7  127591212  127591296   
                   1  358019   chr7  127591033  127591088   
ENST00000000412.8  0  528168  chr12    8941820    8941940   
                   1  528167  chr12    8942415    8942470   
ENST00000000442.11 0  493539  chr11   64315706   64315963   
...                      ...    ...        ...        ...   
ENST00000713920.1  1  649116  chr16    1320462    1320517   
ENST00000713921.1  0  649156  chr16    1324729    1324790   
                   1  649155  chr16    1320462    1320517   
ENST00000713922.1  0  649327  chr16    1314756    1314858   
                   1  649326  chr16    1314321    1314376   

                                                           cds_id score  \
transcript_name                                                           
ENST00000000233.10 0  ENST00000000233.10_cds_0_0_chr7_127591213_f     .   
                   1  ENST00000000233.10_cds_0_0_chr7_127590963_f     .   
ENST00000000412.8  0    ENST00000000412.8_cds_0_0_chr12_8941821_r     .   
                   1    ENST00000000412.8_cds_0_0_chr12_8942416_r     .   
ENST00000000442.11 0  ENST00000000442.11_cds_0_0_chr11_64315707_f     .   
...                                                           ...   ...   
ENST00000713920.1  1    ENST00000713920.1_cds_0_0_chr16_1320438_f     .   
ENST00000713921.1  0    ENST00000713921.1_cds_0_0_chr16_1324730_f     .   
                   1    ENST00000713921.1_cds_0_0_chr16_1320438_f     .   
ENST00000713922.1  0    ENST00000713922.1_cds_0_0_chr16_1314757_f     .   
                   1    ENST00000713922.1_cds_0_0_chr16_1314293_f     .   

                     strand     transcript_name cds_size  
transcript_name                                           
ENST00000000233.10 0      +  ENST00000000233.10       84  
                   1      +  ENST00000000233.10       55  
ENST00000000412.8  0      -   ENST00000000412.8      120  
                   1      -   ENST00000000412.8       55  
ENST00000000442.11 0      +  ENST00000000442.11      257  
...                     ...                 ...      ...  
ENST00000713920.1  1      +   ENST00000713920.1       55  
ENST00000713921.1  0      +   ENST00000713921.1       61  
                   1      +   ENST00000713921.1       55  
ENST00000713922.1  0      +   ENST00000713922.1      102  
                   1      +   ENST00000713922.1       55  

[213979 rows x 9 columns]

### Как получать контекст последовательности с помощью pyfaidx

In [ ]:
from pyfaidx import Fasta

https://pypi.org/project/pyfaidx/

Загружаем нашу фасту с транскриптами, на этом этапе создается индекс fai.  

Дополнительно кастомизируем ключи, т.к. в файле траснкриптов ключи выглядят так:  
`ENST00000353265.8|ENSG00000178184.16|OTTHUMG00000132922.4|OTTHUMT00000256435.3|PARD6G-201|PARD6G|3836|protein_coding|`  
А нам нужна только их первая часть.

Тут бесячий момент: надо переархивировать файл с транскриптами в BGZF  
_Compressed FASTA is only supported in BGZF format. Use the samtools bgzip utility (instead of gzip) to compress your FASTA._  

<font color='brown'> Похоже, важно иметь версию >0.7.0, т.к. ниже неё какие-то проблемы: несмотря на перевод файла в BGZF формат, ошибка о неправильном инпуте не пропадала. А после апдейта до v.0.8.1.1 всё сработало.

Интересующий транскрипт и позиция:

Индексация с 0, поэтому чтобы получить позицию 34, вводим 33:

Утилита работает как словарь, тогда по примеру из документации:

Тут вызвали комплементарную позицию.

Проверила глазами по таблице, всё сходится.

Ниже похожий способ, но с помощью `merge`.

В датафрейме `merged_clinvar_and_ben` у нас произошло слияние двух дф на основе совпадения значений в колонках 'Chr', 'Position', 'Ref', 'Alt'. Все строки из `clinvar_final_pat`, у которых не нашлось совпадений в `ben_df` в 'Chr', 'Position', 'Ref', 'Alt', отброшены.  
Также в смёрдженном датафрейме нем есть колонка `'_merge'`, которая показывает, в какой из исходных таблиц содержится строка: `'both'` для совпадающих строк, `'left_only'` для строк, найденных только в `ben_df`, и `'right_only'` для строк, найденных только в `clinvar_final_pat`.

In [97]:
# merged_clinvar_and_ben = ben_df.merge(clinvar_final_pat, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', indicator=True)
# merged_clinvar_and_ben
# 116963 rows × 29 columns

Мы оставляем только те строки, которые присутствуют в `ben_df`, но отсутствуют в `clinvar_final_pat`. Т.е. оставляем только `left_only`.

In [98]:
# ben_df_filtered = merged_clinvar_and_ben[merged_clinvar_and_ben['_merge'] == 'left_only'].drop(columns='_merge')
# ben_df_filtered
# 114271 rows × 28 columns

Избавляемся от артефактов после пересечения датафреймов и наводим красоту.

In [99]:
# ben_df_filtered = ben_df_filtered.drop(columns=['Consequence_y', 'SYMBOL', 'cDNA_position_y', 
#                                                 'Gene',	'Feature_type', 'Feature', 'BIOTYPE', 'cDNA_position_y', 'CANONICAL',
#                                                'ID', 'CLNSIG', 'CLNVC', 'GENEINFO', 'MC'])

# ben_df_filtered = ben_df_filtered.rename(columns={'cDNA_position_x': 'cDNA_position', 'Consequence_x': 'Consequence'})
# ben_df_filtered
# 114271 rows × 15 columns